In [2]:
# Import the two main libraries
import warnings
warnings.filterwarnings("ignore")

import datetime
import json
import pickle
import sys
import time

import pandas as pd
import requests
from bs4 import BeautifulSoup  # to process html
from tqdm import tqdm

In [11]:
#get all the links of the articles

daily_mail_links = []
for page in tqdm(range(1,21)):

    url = 'https://www.dailymail.co.uk/news/russia-ukraine-conflict/index.html?page='+str(page)
    page = requests.get(url)
    soup = BeautifulSoup(page.text, "html.parser")
    
    links = soup.find_all('h2',{'linkro-darkred'})
    
    for link in links:
        daily_mail_links.append(link.find('a')['href'].strip())
        
    

100%|██████████| 20/20 [00:33<00:00,  1.66s/it]


In [14]:
file_name = "daily_mail_links.pkl"

open_file = open(file_name, "wb")
pickle.dump(daily_mail_links, open_file)
open_file.close()

In [185]:
DailyArticles = pd.DataFrame({'title': pd.Series([], dtype='string'),
                             'author': pd.Series([], dtype='string'),
                             'date': pd.Series([], dtype='string'),
                             'text': pd.Series([], dtype='string'),
                             })


In [145]:
for i,link in enumerate(tqdm(daily_mail_links)):
    
    page = requests.get(link)
    soup = BeautifulSoup(page.text, "html.parser")
    
    #get title
    title = soup.find_all("div", {"class": "article-text wide heading-tag-switch"})
    
    if len(title) == 0:
        continue
    else:
        title = title[0].find('h2').get_text()
    
    #get author
    authors = soup.find_all('a', {'class': 'author'})
    art_authors = authors[0].get_text()
    for i in range(1,len(authors)):
        art_authors = art_authors +", " + authors[1].get_text()
        
        
    #get date
    date = soup.find_all('span', {'class': 'article-timestamp article-timestamp-published'})
    if len(date) > 0:
        date = date[0].find('time').get_text().split(',')[1].strip()
    else:
        date = soup.find_all('span', {'class': 'article-timestamp article-timestamp-updated'})[0].find('time').get_text().split(',')[1].strip()
        
    date = pd.to_datetime(date).strftime('%d/%m/%y')
    
    #get text
    text = ''
    for p in soup.find_all('p',{'class' : 'mol-para-with-font'}):
        text += p.text
    
    entry = {'link': link,'title': title, 'author': art_authors,'date': date,'text':text}
    DailyArticles = pd.concat([DailyArticles, pd.DataFrame(entry, index=[0])], ignore_index=True)
    
    

100%|██████████| 1600/1600 [31:27<00:00,  1.18s/it]


In [22]:
#changing the user agent in order to get the json fil with the comments
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1.2 Safari/605.1.15'
    }  



In [23]:
DailyArticles["comments"] = 0
articleNumber = []

for i, link in enumerate(tqdm(DailyArticles["link"])):
    comments = []
    articleNumber = link.split('article-')[1].split('/')[0]
    data = requests.get(
        'https://www.dailymail.co.uk/reader-comments/p/asset/readcomments/'
        + str(articleNumber)
        + '?max=1000&sort=voteRating&order=desc',headers = headers)

    data = json.loads(data.text)

    for comment in data["payload"]['page']:
        comments.append(comment["message"])

    DailyArticles["comments"][i] = comments

100%|██████████| 1277/1277 [05:54<00:00,  3.60it/s]


In [24]:
DailyArticles.to_parquet("DailyMail.parquet.snappy", engine="fastparquet",index = False)